In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
%matplotlib inline
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

# Try scratched Vgg

In [3]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [4]:
SHAPE = (64, 64)
CHANNEL = 3
    
data = generate_data_set_for_training(shape=SHAPE, channel=CHANNEL)
# Extract Train, val from data
X_train = data['X_train']
X_val = data['X_val']
# Format as the input of VGG
y_train = list(data['y_train'].T)
y_val = list(data['y_val'].T)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [ ]:
_,row, col,channel = X_train.shape
digLen = 5 # including category 0
numDigits = 11
epochs = 50
batch_size = 64

vgg16Model = VGG16(include_top = False,
                   weights = None)
vgg16Model.summary()
ptInput = keras.Input(shape = (row,col,channel), name  = 'vgg16Scratch')
vgg16 = vgg16Model(ptInput)

# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(input)
# vgg16 = Conv2D(64,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(128,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(256,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)
#
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = Conv2D(512,(3, 3), activation ='relu', padding='same')(vgg16)
# vgg16 = MaxPooling2D(pool_size=(2, 2))(vgg16)

vgg16 = Flatten()(vgg16)
vgg16 = Dense(1024, activation='relu')(vgg16)
vgg16 = Dense(1024, activation='relu')(vgg16)
# vgg16 = Dense(1000, activation='relu')(vgg16)
vgg16 = Dropout(0.5)(vgg16)

numd_SM = Dense(digLen,    activation='softmax',name = 'num')(vgg16)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(vgg16)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(vgg16)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(vgg16)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(vgg16)
numB_SM = Dense(2,         activation='softmax',name = 'nC')(vgg16)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM, numB_SM]

vgg16 = keras.Model(inputs = ptInput, outputs = out)

callback = []
optim = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/vgg16.classifier.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 3,
                                              cooldown= 0,
                                              min_lr = 0.000001)
# tb = keras.callbacks.TensorBoard(log_dir='logs', write_graph=True, write_images=True)
# es = keras.callbacks.EarlyStopping(monitor= 'val_loss',
#                                    min_delta=0.00000001,
#                                    patience=5,
#                                    verbose=1,
#                                    mode='auto')
#callback.append(es)

In [6]:
callback.append(checkpointer)
callback.append(reduce_lr)
vgg16.summary()

vgg16.compile(loss = 'sparse_categorical_crossentropy',
              optimizer= optim,
              metrics=  ['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
vgg16Scratch (InputLayer)       (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    vgg16Scratch[0][0]               
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2048)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         2098176     flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (D

In [7]:
vgg16History = vgg16.fit(x = X_train,
                         y = y_train,
                         batch_size = batch_size,
                         epochs=epochs,
                         verbose=1,
                         shuffle = True,
                         validation_data = (X_val, y_val),
                         callbacks = callback)

print(vgg16History.history.keys())
modName = 'vgg16_Scratch'
print(vgg16History.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 92s 1ms/step - loss: 5.2512 - num_loss: 1.1180 - dig1_loss: 1.4738 - dig2_loss: 1.2648 - dig3_loss: 0.5829 - dig4_loss: 0.1299 - nC_loss: 0.6819 - num_acc: 0.6104 - dig1_acc: 0.6125 - dig2_acc: 0.7112 - dig3_acc: 0.8972 - dig4_acc: 0.9848 - nC_acc: 0.6088 - val_loss: 5.1057 - val_num_loss: 1.0941 - val_dig1_loss: 1.4415 - val_dig2_loss: 1.2369 - val_dig3_loss: 0.5630 - val_dig4_loss: 0.1020 - val_nC_loss: 0.6681 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00001: loss improved from inf to 5.25117, saving model to saved_models/vgg16.classifier.hdf5
Epoch 2/50
69076/69076 [==============================] - 84s 1ms/step - loss: 5.1248 - num_loss: 1.0997 - dig1_loss: 1.4483 - dig2_loss: 1.2394 - dig3_loss: 0.5602 - dig4_loss: 0.1084 - nC_loss: 0.6687 - num_acc: 0.6136 - dig1_acc: 0.6146 -


Epoch 00012: loss improved from 5.10561 to 5.10446, saving model to saved_models/vgg16.classifier.hdf5
Epoch 13/50
69076/69076 [==============================] - 84s 1ms/step - loss: 5.1044 - num_loss: 1.0961 - dig1_loss: 1.4428 - dig2_loss: 1.2345 - dig3_loss: 0.5565 - dig4_loss: 0.1070 - nC_loss: 0.6675 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9858 - nC_acc: 0.6136 - val_loss: 5.1020 - val_num_loss: 1.0942 - val_dig1_loss: 1.4398 - val_dig2_loss: 1.2355 - val_dig3_loss: 0.5631 - val_dig4_loss: 0.1012 - val_nC_loss: 0.6681 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00013: loss improved from 5.10446 to 5.10441, saving model to saved_models/vgg16.classifier.hdf5
Epoch 14/50
69076/69076 [==============================] - 83s 1ms/step - loss: 5.1049 - num_loss: 1.0965 - dig1_loss: 1.4430 - dig2_loss: 1.2342 - dig3_loss: 0.5565 - dig4_loss: 0.1


Epoch 00024: loss improved from 5.10238 to 5.09966, saving model to saved_models/vgg16.classifier.hdf5
Epoch 25/50
69076/69076 [==============================] - 84s 1ms/step - loss: 5.0986 - num_loss: 1.0954 - dig1_loss: 1.4414 - dig2_loss: 1.2328 - dig3_loss: 0.5548 - dig4_loss: 0.1070 - nC_loss: 0.6672 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9858 - nC_acc: 0.6136 - val_loss: 5.1001 - val_num_loss: 1.0937 - val_dig1_loss: 1.4394 - val_dig2_loss: 1.2353 - val_dig3_loss: 0.5626 - val_dig4_loss: 0.1012 - val_nC_loss: 0.6679 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00025: loss improved from 5.09966 to 5.09858, saving model to saved_models/vgg16.classifier.hdf5
Epoch 26/50
69076/69076 [==============================] - 85s 1ms/step - loss: 5.1004 - num_loss: 1.0959 - dig1_loss: 1.4418 - dig2_loss: 1.2330 - dig3_loss: 0.5555 - dig4_loss: 0.1


Epoch 00036: loss did not improve from 5.09794
Epoch 37/50
69076/69076 [==============================] - 83s 1ms/step - loss: 5.0994 - num_loss: 1.0957 - dig1_loss: 1.4416 - dig2_loss: 1.2328 - dig3_loss: 0.5552 - dig4_loss: 0.1068 - nC_loss: 0.6673 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9858 - nC_acc: 0.6136 - val_loss: 5.0999 - val_num_loss: 1.0936 - val_dig1_loss: 1.4393 - val_dig2_loss: 1.2354 - val_dig3_loss: 0.5625 - val_dig4_loss: 0.1012 - val_nC_loss: 0.6679 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00037: loss did not improve from 5.09794
Epoch 38/50
69076/69076 [==============================] - 83s 1ms/step - loss: 5.0995 - num_loss: 1.0955 - dig1_loss: 1.4413 - dig2_loss: 1.2327 - dig3_loss: 0.5556 - dig4_loss: 0.1071 - nC_loss: 0.6672 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9

69076/69076 [==============================] - 83s 1ms/step - loss: 5.0990 - num_loss: 1.0956 - dig1_loss: 1.4413 - dig2_loss: 1.2327 - dig3_loss: 0.5554 - dig4_loss: 0.1069 - nC_loss: 0.6671 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9858 - nC_acc: 0.6136 - val_loss: 5.0999 - val_num_loss: 1.0936 - val_dig1_loss: 1.4393 - val_dig2_loss: 1.2354 - val_dig3_loss: 0.5625 - val_dig4_loss: 0.1012 - val_nC_loss: 0.6679 - val_num_acc: 0.6118 - val_dig1_acc: 0.6127 - val_dig2_acc: 0.7106 - val_dig3_acc: 0.8959 - val_dig4_acc: 0.9865 - val_nC_acc: 0.6118

Epoch 00049: loss did not improve from 5.09794
Epoch 50/50
69076/69076 [==============================] - 83s 1ms/step - loss: 5.0990 - num_loss: 1.0954 - dig1_loss: 1.4413 - dig2_loss: 1.2329 - dig3_loss: 0.5553 - dig4_loss: 0.1070 - nC_loss: 0.6671 - num_acc: 0.6136 - dig1_acc: 0.6146 - dig2_acc: 0.7120 - dig3_acc: 0.8980 - dig4_acc: 0.9858 - nC_acc: 0.6136 - val_loss: 5.0999 - val_num_loss: 1.09

In [ ]:
%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance

evaluate_model_performance(fitted_model=vgg16History, 
                           model_name = modName,
                           data=data,
                           model=vgg16
                          )